In [18]:
from drugrelink import pairs


In [3]:
list1,list2,list3,list4 =pairs.test_pairs(validation_path='/home/lxu/drugrelink/data/validation-statuses.tsv',train_path='/home/lxu/drugrelink/data/transformed-features.tsv.bz2',symptomatic_path='/home/lxu/drugrelink/data/probabilities.tsv')

In [11]:
len(list1[0][])

209168

In [2]:
!head /home/lxu/drugrelink/data/validation-statuses.tsv

compound_id	compound_name	disease_id	disease_name	n_trials	status_trials	status_drugcentral
DB01048	Abacavir	DOID:10652	Alzheimer's disease	0	0	0
DB01048	Abacavir	DOID:9206	Barrett's esophagus	0	0	0
DB01048	Abacavir	DOID:13241	Behcet's disease	0	0	0
DB01048	Abacavir	DOID:11949	Creutzfeldt-Jakob disease	0	0	0
DB01048	Abacavir	DOID:8778	Crohn's disease	0	0	0
DB01048	Abacavir	DOID:11555	Fuchs' endothelial dystrophy	0	0	0
DB01048	Abacavir	DOID:11119	Gilles de la Tourette syndrome	0	0	0
DB01048	Abacavir	DOID:12361	Graves' disease	0	0	0
DB01048	Abacavir	DOID:2986	IgA glomerulonephritis	0	0	0


In [ ]:
def find_difference(l1,l2):
    pairs = [[],[]]
    for i in range(len(l1[0])):
        for y in range (len(l2[0])):
            if l1[0][i] == l2[0][y] and l1[1][i] != l2[1][y]:
                pairs[0].append(l2[0][y])
                pairs[1].append(l2[1][y])
                
    return pairs
            

positive4L2 = find_difference(list1,list2)
len(positive4L2)

In [ ]:
positive4L3 = find_difference(list1,list3)
print(len(positive4L3))

In [ ]:
positive4L4 = find_difference(list1,list4)
print(len(positive4L4))

In [ ]:
def find_negative(l1,l2):
    pairs =[]
    for i in range(len(l1[0])):
        for y in range(len(l2[0])):
            if l1[1][i] == 1 and 

In [25]:
from drugrelink import pairs
df = pairs.data_non_overlap(validation_path='/home/lxu/drugrelink/data/validation-statuses.tsv',train_path='/home/lxu/drugrelink/data/transformed-features.tsv.bz2',symptomatic_path='/home/lxu/drugrelink/data/probabilities.tsv')

UnboundLocalError: local variable 'data_df' referenced before assignment

In [30]:
from drugrelink import pairs
list1,list2,list3,list4 =pairs.test_pairs(validation_path='/home/lxu/drugrelink/data/validation-statuses.tsv',train_path='/home/lxu/drugrelink/data/transformed-features.tsv.bz2',symptomatic_path='/home/lxu/drugrelink/data/probabilities.tsv')

TypeError: append() takes exactly one argument (2 given)